In [1]:
'065676'

'065676'

In [1]:
# ! pip install jugaad_data, nsetools

import backtrader

from helpers.candlestick import CandlePattern
from helpers.datahandler import *
from helpers.stock_analyser import *

from multiprocessing import Pool

# Defaults

In [29]:
CP = CandlePattern()

current_date = date.today()

pd.set_option('display.max_rows', 100)

# Code

In [3]:
class DataHandler:
    def __init__(self, data_path = './data', check_fresh = True):
        self.present = date.today()
        self.week_num = self.present.strftime("%W")
        
        self.data_path = data_path
        
        if check_fresh:
            self.__fresh()
        self.data = self.read_data()
        self.all_stocks = self.read_data()['all_stocks']
        
         
    def __fresh(self,):
        files = listdir(self.data_path)
        if not len(files):
            raise Exception(f"No CSV data files present at {self.data_path} Download new data for analysis")

        self.data = self.read_data()
        for file in files:
            key, _ , _ = file.split('_')
            self.data['all_stocks'][key] = file
        self.update_data(self.data)
  
    
    def read_data(self, path = './', file = 'data.json'):
        '''
        Write the data in json file
        args:
            path: Path of the directory
            File: Json Filename
        '''
        with open(join(path,file)) as f:
            return json.load(f)


    def update_data(self, updated_data:dict, path:str = './', file:str = 'data.json'):
        '''
        Update the data in the json file
        args:
            updated_data: Dictonary you want to update
            path: Path of the directory
            File: Json Filename
        '''
        with open(join(path,file), 'w') as f:
            json.dump(updated_data,f)
            return True 
    
    
    def open_live_stock_data(self,name:str):
        '''
        Open the fresh stock from the market
        args:
            name: ID of the stock given
        '''
        drop = ['SERIES','PREV. CLOSE','VWAP','VOLUME','VALUE','NO OF TRADES']
        return stock_df(symbol=name, from_date = self.present - timedelta(days = 600), to_date = self.present, series="EQ").drop(drop,axis=1)
    
    
    def open_downloaded_stock(self,name:str):
        '''
        Open the Individual stock based on it's Official Term
        args:
            name: Name / ID given to the stock. Example, Infosys is "INFY"
        returns: DataFrame of that stock
        '''
        return pd.read_csv(join(self.data_path,self.all_stocks[name]))

In [4]:
class AnalyseStocks(DataHandler):
    def __init__(self, check_fresh = True):
        '''
        args:
            path: Path where all the stock files are saved
        '''
        super().__init__(check_fresh = check_fresh)
        self.registered_stocks = self.read_data()['registered_stocks']
        self.colors = self.read_data()['colors']
        self.rising = {}
    

    def is_ma_eligible(self, df, limit:float, mv = 44, names:tuple = ('DATE','OPEN','CLOSE','LOW','HIGH')):
        '''
        Find the Positive Stocks which are about to rise on the Moving average line
        args:
            df: DataFrame
            limit: Limit difference between average and low/high threshold for selecting stocks
            mv: Moving Average to Consider
            names: Tuple of column names showing ('DATE','OPEN','CLOSE','LOW','HIGH')
        '''
        Date, Open, Close, Low, High = names
        Average = f'{str(mv)}-SMA'
        
        stocks = df.sort_index(ascending=False,) # Sort the values else Moving average for new values will be empty
        stocks[Average] = stocks[Close].rolling(mv, min_periods = 1).mean()
        
        last_traded = stocks.iloc[-1,:]
        low, high, open_ , close, avg, symbol = last_traded[Low],last_traded[High],last_traded[Open],last_traded[Close], last_traded[Average], last_traded['SYMBOL']
        
        if close < avg or close < open_ : # if red candle or below Average Line, Discard
            return False
        
        diff = min(abs(low - avg), (abs(high - avg)), abs(open_ - avg), abs(close - avg))
        if diff <= limit:
            self.rising[symbol] = stocks.iloc[-mv//2:-1,-1].mean() < avg  # whether the moving average is Upward or Downward according to last 44 days moving averages
            return {symbol : round(diff,2)}

        return False
    
    
    def BollingerBands(self,df, mv:int = 44):
        '''
        Calculate the Upper and Lower Bollinger Bands Given on a Moving Average Line
        args:
            df: Stocks Data
            mv: Moving Average Line value
        '''
        ticker = df.sort_index(ascending=False)

        sma = ticker['CLOSE'].rolling(window = mv).mean()
        std = ticker['CLOSE'].rolling(window = mv).std(0)

        upper_bb = sma + std * 2
        lower_bb = sma - std * 2

        ticker['Upper Bollinger'] = upper_bb
        ticker['Lower Bollinger'] = lower_bb

        return ticker.sort_index(ascending=True)
    
    
    def Ichimoku_Cloud(self,df):
        '''
        Get the values of Lines for Ichimoku Cloud
        args:
            df: Dataframe
        '''
        d = df.sort_index(ascending=False)

        # Tenkan-sen (Conversion Line): (9-period high + 9-period low)/2))
        period9_high = d['HIGH'].rolling(window=9).max()
        period9_low = d['LOW'].rolling(window=9).min()
        tenkan_sen = (period9_high + period9_low) / 2


        # Kijun-sen (Base Line): (26-period high + 26-period low)/2))
        period26_high = d['HIGH'].rolling(window=26).max()
        period26_low = d['LOW'].rolling(window=26).min()
        kijun_sen = (period26_high + period26_low) / 2

        # Senkou Span A (Leading Span A): (Conversion Line + Base Line)/2))
        senkou_span_a = ((tenkan_sen + kijun_sen) / 2).shift(26)

        # Senkou Span B (Leading Span B): (52-period high + 52-period low)/2))
        period52_high = d['HIGH'].rolling(window=52).max()
        period52_low = d['LOW'].rolling(window=52).min()
        senkou_span_b = ((period52_high + period52_low) / 2).shift(26)

        # The most current closing price plotted 22 time periods behind (optional)
        chikou_span = d['CLOSE'].shift(-26) # Given at Trading View.

        d['blue_line'] = tenkan_sen
        d['red_line'] = kijun_sen
        d['cloud_green_line_a'] = senkou_span_a
        d['cloud_red_line_b'] = senkou_span_b
        d['lagging_line'] = chikou_span
        return d.sort_index(ascending=True)
    
    
    def _is_ichi(self,df, index:str = 'nifty_50'):
        '''
        Get All available IchiMoku available stocks
        args:
            index: Index to Open From
        '''
        count  = 0
        current = df.iloc[0,:]
        if current['cloud_green_line_a'] < current['LOW'] and current['cloud_red_line_b'] < current['LOW']: # Cloud Below
            count += 1
        if df.loc[26,'lagging_line'] > df.loc[26,'HIGH']: # Lagging Line
            count += 1
        if df.loc[1,'blue_line'] <= df.loc[1,'red_line'].min() and current['blue_line'] >= current['red_line']: # Cross Over
            count += 1
        
        return count
    
    
    def update_eligible(self, limit:float):
        '''
        Save all Eligible stocks for the current week
        args:
            limit: Distance between MA line and the Min(open,close,low,high)
        '''
        self.eligible = {}
        for key in self.registered_stocks:
            result = self.is_ma_eligible(self.open_downloaded_stock(key), limit = limit)
            if result:
                self.eligible.update(result)
        return self.eligible
    
    
    
            
    
    def plot_candlesticks(self,df, names = ('DATE','OPEN','CLOSE','LOW','HIGH'), mv:list = [44,100,200]):
        '''
        Plot a candlestick on a given dataframe
        args:
            df: DataFrame
            names: Tuple of column names showing ('DATE','OPEN','CLOSE','LOW','HIGH')
            mv: Moving Averages
        '''
        stocks = df.copy()
        Date, Open, Close, Low, High = names
        colors = sample(self.colors,len(mv))
        stocks.sort_index(ascending=False, inplace = True)  # Without reverse, recent rolling mean will be either NaN or equal to the exact value
    

        candle = go.Figure(data = [go.Candlestick(x = stocks[Date], name = 'Trade',
                                                       open = stocks[Open], 
                                                       high = stocks[High], 
                                                       low = stocks[Low], 
                                                       close = stocks[Close]),])
        for i in range(len(mv)):
            stocks[f'{str(mv[i])}-SMA'] = stocks[Close].rolling(mv[i], min_periods = 1).mean()
            candle.add_trace(go.Scatter(name=f'{str(mv[i])} MA',x=stocks[Date], y=stocks[f'{str(mv[i])}-SMA'], 
                                             line=dict(color=colors[i], width=1.1)))

        candle.update_xaxes(
            title_text = 'Date',
            rangeslider_visible = True,
            rangeselector = dict(
                buttons = list([
                    dict(count = 1, label = '1M', step = 'month', stepmode = 'backward'),
                    dict(count = 6, label = '6M', step = 'month', stepmode = 'backward'),
                    dict(count = 1, label = 'YTD', step = 'year', stepmode = 'todate'),
                    dict(count = 1, label = '1Y', step = 'year', stepmode = 'backward'),
                    dict(step = 'all')])))

        candle.update_layout(autosize = False, width = 1400, height = 600,
                             title = {'text': f"{stocks['SYMBOL'][0]} | {self.all_stocks[stocks['SYMBOL'][0]]}",'y':0.97,'x':0.5,
                                      'xanchor': 'center','yanchor': 'top'},
                             margin=dict(l=30,r=30,b=30,t=30,pad=2),
                             paper_bgcolor="lightsteelblue",)

        candle.update_yaxes(title_text = 'Price in Rupees', tickprefix = u"\u20B9" ) # Rupee symbol
        candle.show()

# Download New Files Data

In [5]:
def get_stocks(name):
    try:
        df = stock_df(symbol=name, from_date = current_date - timedelta(days = 600), to_date = current_date, series="EQ").drop(drop,axis=1)
        df['DATE'] = pd.to_datetime(df['DATE'])
        ID, NAME, _ = all_stocks[name].split('_')
        save = f"./data/{ID}_{NAME}_{str(current_date)}.csv"
        df.to_csv(save,index=None)
    except Exception as e:
        print(e)
        pass


 
path = './data'
AS = AnalyseStocks(check_fresh = False)
all_stocks = AS.all_stocks

drop = ['SERIES','PREV. CLOSE','VWAP','VOLUME','VALUE','NO OF TRADES']
current_date = date.today()
stocks = set(all_stocks.keys()) - set([i.split('_')[0] for i in listdir('./data')]) 

pool = Pool(4)
results = pool.map(get_stocks,stocks)
pool.close()
pool.join()

# Investing

In [6]:
class Investing(AnalyseStocks):
    def __init__(self,rolling_mean:int=44):
        '''
        args:
            rolling_mean: Rolling Simple Mean to calulate
        '''
        super().__init__()
        self.rm = rolling_mean
        self._eligible = None
        self.data = self.read_data()
        self.indices = {'nifty_50': 'Nifty 50','nifty_100':'Nifty 100','nifty_200':'Nifty 200','nifty_500':'Nifty 500'}
        self.all_ichi = None
        self.picked = None
        self._old_budget = -1
        self.diff = -1
      

    def _get_all_ichi(self,budget:float, index:str='nifty_500', refit = False):
        '''
        Get all Stocks who are almost perfect for Ichimoku execution
        args:
            budget: Your Budget
            index: Which Index to Search
        '''
        if self.all_ichi and  (not refit):
            return self.all_ichi
        
        all_ichi = {}
        data = self.data[index] if index else self.all_stocks
        for name in data:
            df = self.Ichimoku_Cloud(self.open_downloaded_stock(name))
            count = self._is_ichi(df,index) 
            if count and df.loc[0,'HIGH'] < budget:
                all_ichi[name] = count
                
        self.all_ichi = all_ichi
        return self.all_ichi
    
    
    def highlight_falling(self, s, column:str):
        '''
        Highlight The rows where average is falling
        args:
            s: Series
            column: Column name(s)
        '''
        is_max = pd.Series(data=False, index=s.index)
        is_max[column] = s.loc[column] == True
        return ['' if is_max.any() else 'background-color: #f7a8a8' for v in is_max]
    
    
    def get_index(self,symbol:str):
        '''
        Get the Index of the symbol from nifty 50,100,200,500
        args:
            symbol: Name /  ID od the company on NSE
        '''
        for index in self.indices.keys():
            if symbol in self.data[index]:
                return self.indices[index]
        return 'Other'
    
    
    def calculate(self, budget, High:str = 'HIGH', Close:str = 'CLOSE', delta:float = 1, nifty:str = 'nifty_200', diff = 13, show_only:bool=True):
        '''
        Pick Stocks based on all available and which are within your budget
        args:
            budget: Total available budget. Stocks under this budget will be considered only
            High: Column name which show High
            Close: Column Name which shows last closing price
            delta: Value above the Last Highest Traded Price
            nifty: nifty index to consider
            show_only: If you want to see formatted part only
            diff: Max Allowed Distance between Min(close,open,low,high) and High
        ''' 
        refit = True if (self._old_budget < budget or self.diff != diff) else False
        self._old_budget = budget
        self.diff = diff
        ichi = self._get_all_ichi(budget ,refit = refit)
        
        if (not self._eligible) or refit:
            self._eligible  = self.update_eligible(limit = diff)
            
        keys = set(self._eligible.keys()).intersection(set(self.data[nifty])) if nifty else list(self._eligible.keys()) 
        if not len(keys):
            warnings.warn('No matching Stocks Found. Increase Distance or Nifty Index')
            return None
        
        values = []
        one_can = []
        two_can = []
        three_can = []
        
        for key in keys:
            try:
                df = self.open_downloaded_stock(key)
            except Exception as e:
                    print('Exception Opening: ',key)
                
            if df.loc[0,High] + delta > budget:
                del self._eligible[key]
                
            else:
                values.append(df.iloc[0,:])
                one_can.append(CP.find_name(df.loc[0,'OPEN'],df.loc[0,'CLOSE'],df.loc[0,'LOW'],df.loc[0,'HIGH']))
                two_can.append(CP.double_candle_pattern(df))
                three_can.append(CP.triple_candle_pattern(df))
                
                
        columns = df.columns
        df = pd.DataFrame(values,columns=columns,index = range(len(values)))
        df = df.merge(pd.DataFrame({'SYMBOL':self._eligible.keys(), 'Diff':self._eligible.values()}),on='SYMBOL')
        
        df['Rising'] = df['SYMBOL'].apply(lambda x: self.rising[x]) # Get Rising or Falling
        df['Ichi'] = df['SYMBOL'].apply(lambda x: ichi[x] if ichi.get(x) else 0)
        
        df['Triple Candle'] = three_can
        df['Double Candle'] = two_can
        df['Recent Candle'] = one_can
        
        df['Index'] = df['SYMBOL'].apply(lambda x: self.get_index(x)) # Get Rising or Falling

        self.picked = df.sort_values('Diff',ascending = True)
        if show_only:
            return self.picked.style.apply(self.highlight_falling, column=['Rising'], axis=1) # set style
        else:
            return self.picked
        
        
    def show_full_stats(self, budget, risk, High = 'HIGH', Close = 'CLOSE', delta:float=1, diff:float = 13, nifty:str = 'nifty_500',):
        '''
        Show Extra Stats
        args:
            budget: Total available budget. Stocks under this budget will be considered only
            risk: How much risk you want to take
            High: Column name which show High
            Close: Column Name which shows last closing price
            delta: Value above the Last Highest Traded Price
            nifty: nifty index to consider
            diff: MAx Allowed Difference between Line and the Price
        '''
        self.picked = self.calculate(budget, High, Close, delta, nifty = nifty, diff = diff, show_only = False)
        
        expec_change = []
        max_risk = []
        
         
        if not isinstance(self.picked, pd.DataFrame):
            return 'No match found. Increase Budget / Diff / Risk / Nifty Index '
        
        pic = self.picked.copy()
        for index in pic.index:
            name = pic.loc[index,'SYMBOL']
            result = self.get_particulars(name,budget,risk)
            if result:
                expec_change.append(result['Target %'])
                max_risk.append(result['Max loss on this config'])
            else:
                expec_change.append(np.inf)
                max_risk.append(np.inf)

        pic['Expected Change %'] = expec_change
        pic['Max Config Risk'] = max_risk
        return pic
        
        
    def get_particulars(self, name, budget:float, max_loss_capacity:float, risk_to_reward_ratio:float=2, Low:str = 'LOW', High:str = 'HIGH', delta:float = 0.0033, plot_candle:bool = False):
        '''
        Display the particulars of a trade before buying
        args:
            name: name of the particular stock
            loss_capacity: How much loss you can survive at the end of day PER SHARE. Total capacity will be No of shares * per share loss capacity
            risk_to_reward_ratio: How much profit you want to have. It is twice of loss_capacity per share for 44 Moving average
            budget : How much you have for investing purpose
            Low: Column name which describes LOW of the previous trade
            High =  Column name which describes High of the previous trade
            delta: A min amount above which you'll buy
            plot_candle: Plot the candlestick for the stock
        '''   
        df = self.open_downloaded_stock(name)
        if risk_to_reward_ratio > 2:
            warnings.warn(f"Don't be greedy with risk to reward ratio of {risk_to_reward_ratio}. Stick to system")
            
        if  max_loss_capacity > 0.011 * budget:
            warnings.warn(f"You are risking {round(max_loss_capacity/budget,2)*100}% of your total PORTFOLIO. Going Like this, you'll lose Rs {max_loss_capacity*15} in 15 Trades. Try keeping it less than 1.1% @ Rs {round(budget*0.011,2)}.")
        
        
        buy_delta = df.loc[0,High] * delta  
        sell_delta = min(df.loc[:1,Low].values) * delta
        
        risk = max_loss_capacity # per share
        
        entry = df.loc[0,High] + buy_delta # Last Day MAX + Delta
        stop_loss = min(df.loc[:1,Low].values) - sell_delta # Min of the last 2 
        
        max_loss = entry - stop_loss
        stop_loss_perc = round((max_loss / entry) *100,2)
        diff = entry - stop_loss
        quantity = min(risk // diff , budget // entry)
        profit = risk_to_reward_ratio * diff
        profit_perc = round((profit/entry)*100,2)
        target = round(entry + profit,2)
        
        
        if plot_candle:
            AnalyseStocks().plot_candlesticks(df)
    
        if budget < entry:
            warnings.warn(f"Budget for {name} should be a minimum of Rs. {entry}")
            return None
        
        if quantity < 1:
            r = round(risk + (diff - risk), 2)
            warnings.warn(f"Risk should be atleast {r} for you to afford {name}")
            return None
            
        return {'Buying Price':round(entry,2),'Stop-Loss %': stop_loss_perc,'Target %':profit_perc,'Quantity':quantity,'Stop-Loss Price':stop_loss,'Trigger Price':target,
                'Risk Per Share':round(diff,2),'Profit Per Share':round(profit,2),'Max loss on this config':round(quantity*diff,2),
                'Max Gain on this config': round(quantity*profit,2),'Price To Profit Ratio': round(entry / (diff*2 ),2), 'Index':self.get_index(name),}
    
In = Investing()

In [13]:
df = In.show_full_stats(10000,100.10, diff = 101, nifty='nifty_100')
# df[(df['Rising'] == True) & (df['Expected Change %'] <= 10) & (df['Ichi'] > 0)]
df.head(100)

/home/admin1/anaconda3/envs/finance/lib/python3.7/site-packages/ipykernel_launcher.py:214: UserWarning: Risk should be atleast 107.88 for you to afford SUNPHARMA
/home/admin1/anaconda3/envs/finance/lib/python3.7/site-packages/ipykernel_launcher.py:214: UserWarning: Risk should be atleast 184.18 for you to afford ALKEM
/home/admin1/anaconda3/envs/finance/lib/python3.7/site-packages/ipykernel_launcher.py:214: UserWarning: Risk should be atleast 119.72 for you to afford TORNTPHARM
/home/admin1/anaconda3/envs/finance/lib/python3.7/site-packages/ipykernel_launcher.py:214: UserWarning: Risk should be atleast 129.96 for you to afford TECHM


DATE     OPEN     HIGH      LOW      LTP    CLOSE    52W H    52W L  \
2   2021-07-30   116.95   119.95   116.25   119.00   118.20   122.00    78.10   
13  2021-07-30   752.00   762.00   745.20   755.00   756.25   788.15   404.30   
15  2021-07-30   550.60   562.00   548.05   559.45   557.75   594.85   364.10   
8   2021-07-30  1024.90  1040.00  1015.15  1031.20  1036.75  1139.50   720.90   
14  2021-07-30   587.20   603.65   584.15   601.85   600.70   605.00   462.95   
3   2021-07-30   532.00   553.20   527.85   547.00   546.70   553.20   333.00   
5   2021-07-30  1537.70  1566.00  1521.65  1555.85  1551.35  1599.50   565.00   
11  2021-07-30   702.95   783.75   696.15   773.55   773.95   783.75   452.25   
1   2021-07-30  1421.90  1452.00  1418.40  1427.35  1428.60  1498.95   897.00   
0   2021-07-30   291.50   306.35   290.50   301.45   301.85   306.35    91.20   
10  2021-07-30  1013.00  1049.95  1009.90  1022.30  1024.95  1067.00   676.50   
7   2021-07-30  1598.00  1624.90  1585.00  1601.95  1601.45  1647.15   843.00   
12  2021-07-30  3334.60  3465.00  3325.35  3461.90  3453.60  3558.90  2415.10   
6   2021-07-30  1610.00  1623.40  1606.15  1611.75  1610.50  1623.40   912.10   
9   2021-07-30  3046.20  3105.75  3023.65  3075.00  3071.10  3144.00  2231.85   
4   2021-07-30  1185.00  1237.30  1175.00  1207.70  1209.55  1237.30   642.55   

        SYMBOL   Diff  Rising  Ichi Triple Candle Double Candle  \
2         NTPC   0.26    True     2     V Pattern       Unknown   
13  TATACONSUM   2.58    True     2       Unknown       Unknown   
15         IGL   4.54    True     2       Unknown       Unknown   
8      SBICARD   4.65   False     2       Unknown       Unknown   
14       DABUR   9.19    True     2     V Pattern       Unknown   
3       MARICO  11.03    True     2     V Pattern       Unknown   
5       GRASIM  11.76    True     2       Unknown       Unknown   
11   SUNPHARMA  16.17    True     2       Unknown       Unknown   
1          UBL  20.80    True     1       Unknown       Unknown   
0         VEDL  21.93   False     2       Unknown       Unknown   
10     HCLTECH  29.68    True     2       Unknown       Unknown   
7           LT  54.59    True     2       Unknown       Unknown   
12       ALKEM  83.67    True     2     V Pattern       Unknown   
6         INFY  86.18    True     2       Unknown       Unknown   
9   TORNTPHARM  92.38    True     2       Unknown       Unknown   
4        TECHM  97.90    True     2       Unknown       Unknown   

      Recent Candle      Index  Expected Change %  Max Config Risk  
2   Inverted Hammer   Nifty 50               9.10            98.57  
13           Hammer   Nifty 50               5.70            87.10  
15          Unknown  Nifty 100               7.33            82.61  
8           Unknown  Nifty 100               7.04            73.43  
14          Unknown  Nifty 100               8.38            76.09  
3           Unknown  Nifty 100              10.73            89.29  
5            Hammer   Nifty 50               6.94            54.54  
11          Bullish   Nifty 50                inf              inf  
1   Inverted Hammer  Nifty 100               7.20            52.44  
0           Unknown  Nifty 100              24.50            75.31  
10  Inverted Hammer   Nifty 50              13.50            71.12  
7            Hammer   Nifty 50               6.19            50.49  
12          Bullish  Nifty 100                inf              inf  
6            Hammer   Nifty 50               4.17            67.88  
9   Inverted Hammer  Nifty 100                inf              inf  
4   Inverted Hammer   Nifty 50                inf              inf

In [25]:
def get_RSI(data, window_length:int=14, Close:str='CLOSE', exp:bool=False, return_df:bool = False):
    '''
    Calculate RSI: Relative Strength Index
    args:
        data: Pandas DataFrame
        window_length: Length of Moving Window
        Close: Name of the column which contains the Closing Price
        exp: Whether to use Exponential Moving Average instead of Simple
        return_df: Whether to return the whole Dataframe. NOTE: It'll have the last value removed
    '''
    if data.iloc[0,0] > data.iloc[1,0]: # if the first Date entry [0,0] is > previous data entry [1,0] then it is in descending order, then reverse it for calculation
        data.sort_index(ascending=False, inplace = True)
        
    data['delta'] = data[Close].diff()  # Get the difference in price from previous step
    data = data.iloc[1:,:]
    close = data.loc[:,Close]
    delta = data['delta']  # Get rid of the first row, which is NaN since it did not have a previous 
    up, down = delta.clip(lower=0), delta.clip(upper=0) # Make the positive gains (up) and negative gains (down) Series
    
    if exp: # if Exponential Average based RSI has to be returned
        roll_up1 = up.ewm(span=window_length).mean() # Calculate the Exponential Moving Average
        roll_down1 = down.abs().ewm(span=window_length).mean()
        
        RS1 = roll_up1 / roll_down1 # Calculate the RSI based on EWMA
        RSI = 100.0 - (100.0 / (1.0 + RS1))
        
    else:
        # Calculate the SMA
        roll_up2 = up.rolling(window_length).mean()
        roll_down2 = down.abs().rolling(window_length).mean()

        # Calculate the RSI based on SMA
        RS2 = roll_up2 / roll_down2
        RSI = 100.0 - (100.0 / (1.0 + RS2))
        
    
    data['RSI'] = RSI
    data.sort_index(ascending=True, inplace = True)
    
    if return_df:
        return data
        
    else: return RSI.iloc[0]

In [33]:
df = stock_df(symbol='SBIN', from_date = current_date - timedelta(days = 600), to_date = current_date, series="EQ")
get_RSI(df, return_df= True, exp=False,Close = 'PREV. CLOSE')

/home/admin1/anaconda3/envs/finance/lib/python3.7/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
/home/admin1/anaconda3/envs/finance/lib/python3.7/site-packages/pandas/core/frame.py:6397: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  key,


DATE SERIES    OPEN    HIGH     LOW  PREV. CLOSE     LTP   CLOSE  \
0   2021-08-02     EQ  434.75  436.70  431.80       431.80  434.35  434.90   
1   2021-07-30     EQ  441.50  444.40  430.70       441.55  432.20  431.80   
2   2021-07-29     EQ  426.00  443.60  425.50       425.50  443.10  441.55   
3   2021-07-28     EQ  430.00  430.50  421.05       429.95  424.80  425.50   
4   2021-07-27     EQ  424.15  433.25  424.00       423.30  429.20  429.95   
..         ...    ...     ...     ...     ...          ...     ...     ...   
404 2019-12-18     EQ  335.00  335.55  325.80       332.90  327.20  326.95   
405 2019-12-17     EQ  333.25  335.75  330.65       331.85  333.55  332.90   
406 2019-12-16     EQ  335.95  337.25  330.80       332.55  332.30  331.85   
407 2019-12-13     EQ  325.20  333.45  325.00       321.85  333.15  332.55   
408 2019-12-12     EQ  314.00  322.95  313.55       312.80  321.45  321.85   

       VWAP   52W H   52W L    VOLUME         VALUE  NO OF TRADES SYMBOL  \
0    434.63  444.40  175.50  16926025  7.356523e+09        175314   SBIN   
1    434.61  444.40  175.50  35196813  1.529676e+10        285719   SBIN   
2    435.48  443.60  175.50  35263606  1.535664e+10        332592   SBIN   
3    424.98  441.95  175.50  17636515  7.495132e+09        188398   SBIN   
4    430.00  441.95  175.50  24119619  1.037149e+10        199224   SBIN   
..      ...     ...     ...       ...           ...           ...    ...   
404  328.80  373.80  244.35  26449075  8.696406e+09        196530   SBIN   
405  333.79  373.80  244.35  26734601  8.923796e+09        184237   SBIN   
406  333.33  373.80  244.35  35027613  1.167570e+10        245635   SBIN   
407  329.84  373.80  244.35  40955226  1.350885e+10        252185   SBIN   
408  317.89  373.80  244.35  37804216  1.201763e+10        234427   SBIN   

     delta        RSI  
0    -9.75  55.812274  
1    16.05  64.202658  
2    -4.45  43.006660  
3     6.65  50.195312  
4    -5.60  40.147524  
..     ...        ...  
404   1.05        NaN  
405  -0.70        NaN  
406  10.70        NaN  
407   9.05        NaN  
408  -0.60        NaN  

[409 rows x 17 columns]

In [71]:
def get_RSI(self, data, periods:int = 14, Close:str = 'CLOSE', ema:bool = True, return_df:bool = False):
    '''
    Calculate RSI: Relative Strength Index
    args:
        data: Pandas DataFrame
        periods: Length of Moving Window
        Close: Name of the column which contains the Closing Price
        ema: Whether to use Exponential Moving Average instead of Simple
        return_df: Whether to return the whole Dataframe. NOTE: It'll have the last value removed
    '''
    df = data.copy()
    if df.iloc[0,0] > df.iloc[1,0]: # if the first Date entry [0,0] is > previous data entry [1,0] then it is in descending order, then reverse it for calculation
        df.sort_index(ascending=False, inplace = True)

    close_delta = df['CLOSE'].diff()

    # Make two series: one for lower closes and one for higher closes
    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)

    if ema: # Use exponential moving average
        ma_up = up.ewm(com = periods - 1, min_periods = periods).mean()
        ma_down = down.ewm(com = periods - 1, min_periods = periods).mean()
        
    else: # Use simple moving average
        ma_up = up.rolling(window = periods,).mean()
        ma_down = down.rolling(window = periods,).mean()

    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))
    df['RSI'] = rsi
    df.sort_index(ascending=True, inplace = True)

    if return_df:
        return df

    return df.iloc[0,-1]

In [72]:
df = In.open_live_stock_data('SBIN')
get_RSI(df)

56.12356112456224